# Part II: LoRA Fine-tuning Using NeMo Customizer

This notebook covers the following:

0. [Prerequisites: Configurations, Health Checks, and Namespaces](#step-0)
1. [Upload Data to NeMo Datastore](#step-1)
2. [LoRA Customization with NeMo Customizer](#step-2)
3. [Running Inference on the Customized Model with NVIDIA NIM](#step-3)

In [1]:
import os
import json
import random
import requests
from openai import OpenAI
from nemo_microservices import NeMoMicroservices

<a id="step-0"></a>
## Prerequisites: Configurations, Health Checks, and Namespaces

Before you proceed, make sure that you completed the first notebook on data preparation to obtain the assets required to follow along.

### Configure NeMo Microservices Endpoints

This section includes importing required libraries, configuring endpoints, and performing health checks to ensure that the NeMo Data Store, NIM, and other services are running correctly.

In [2]:
from config import *

# # Initialize NeMo Microservices SDK client
# nemo_client = NeMoMicroservices(
#     base_url=NEMO_URL,
#     inference_base_url=NIM_URL,
# )

# Separate SDK clients per service
entity_client = NeMoMicroservices(base_url=NEMO_ENTITY_STORE_URL, inference_base_url=NIM_URL)
customizer_client = NeMoMicroservices(base_url=NEMO_CUSTOMIZER_URL, inference_base_url=NIM_URL)
evaluator_client = NeMoMicroservices(base_url=NEMO_EVALUATOR_URL, inference_base_url=NIM_URL)
guardrails_client = NeMoMicroservices(base_url=NEMO_GUARDRAILS_URL, inference_base_url=NIM_URL)

In [3]:
print(f"Data Store endpoint: {NEMO_DATA_STORE_URL}")
print(f"Entity Store endpoint: {NEMO_ENTITY_STORE_URL}")
print(f"Customizer endpoint: {NEMO_CUSTOMIZER_URL}")
print(f"Evaluator endpoint: {NEMO_EVALUATOR_URL}")
print(f"NIM endpoint: {NIM_URL}")
print(f"Namespace: {NMS_NAMESPACE}")
print(f"Base Model for Customization: {BASE_MODEL}@{BASE_MODEL_VERSION}")

Data Store endpoint: http://nemo-data-store.nemo.svc.cluster.local:3000
Entity Store endpoint: http://nemo-entity-store.nemo.svc.cluster.local:8000
Customizer endpoint: http://nemo-customizer.nemo.svc.cluster.local:8000
Evaluator endpoint: http://nemo-evaluator.nemo.svc.cluster.local:7331
NIM endpoint: http://nemo-nim-proxy.nemo.svc.cluster.local:8000
Namespace: xlam-tutorial-ns
Base Model for Customization: meta/llama-3.2-1b-instruct@v1.0.0+L40


### Configure Path to Prepared data

The following code sets the paths to the prepared dataset files.

In [4]:
# Path where data preparation notebook saved finetuning and evaluation data
DATA_ROOT = os.path.join(os.getcwd(), "data")
CUSTOMIZATION_DATA_ROOT = os.path.join(DATA_ROOT, "customization")
VALIDATION_DATA_ROOT = os.path.join(DATA_ROOT, "validation")
EVALUATION_DATA_ROOT = os.path.join(DATA_ROOT, "evaluation")

# Sanity checks
train_fp = f"{CUSTOMIZATION_DATA_ROOT}/training.jsonl"
assert os.path.exists(train_fp), f"The training data at '{train_fp}' does not exist. Please ensure that the data was prepared successfully."

val_fp = f"{VALIDATION_DATA_ROOT}/validation.jsonl"
assert os.path.exists(val_fp), f"The validation data at '{val_fp}' does not exist. Please ensure that the data was prepared successfully."

test_fp = f"{EVALUATION_DATA_ROOT}/xlam-test-single.jsonl"
assert os.path.exists(test_fp), f"The test data at '{test_fp}' does not exist. Please ensure that the data was prepared successfully."

### Resource Organization Using Namespace

You can use a [namespace](https://docs.nvidia.com/nemo/microservices/latest/manage-entities/namespaces/index.html) to isolate and organize the artifacts in this tutorial.

#### Create Namespace

Both Data Store and Entity Store use namespaces. The following code creates namespaces for the tutorial.

In [5]:
def create_namespaces(nemo_client, ds_host, namespace):
    # Create namespace in Entity Store
    try:
        namespace_obj = nemo_client.namespaces.create(id=namespace)
        print(f"Created namespace in Entity Store: {namespace_obj.id}")
    except Exception as e:
        # Handle if namespace already exists
        if "409" in str(e) or "422" in str(e):
            print(f"Namespace {namespace} already exists in Entity Store")
        else:
            raise e

    # Create namespace in Data Store (still using requests as SDK doesn't cover Data Store)
    nds_url = f"{ds_host}/v1/datastore/namespaces"
    resp = requests.post(nds_url, data={"namespace": namespace})
    assert resp.status_code in (200, 201, 409, 422), \
        f"Unexpected response from Data Store during namespace creation: {resp.status_code}"
    print(f"Data Store namespace creation response: {resp}")

create_namespaces(nemo_client=entity_client, ds_host=NEMO_DATA_STORE_URL, namespace=NMS_NAMESPACE)

HTTP Request: POST http://nemo-entity-store.nemo.svc.cluster.local:8000/v1/namespaces "HTTP/1.1 200 OK"


Created namespace in Entity Store: xlam-tutorial-ns
Data Store namespace creation response: <Response [201]>


#### Verify Namespaces

The following [Data Store API](https://docs.nvidia.com/nemo/microservices/latest/api/datastore.html) and [Entity Store API](https://docs.nvidia.com/nemo/microservices/latest/api/entity-store.html) list the namespace created in the previous cell.

In [6]:
# Verify Namespace in Data Store (using requests as SDK doesn't cover Data Store)
response = requests.get(f"{NEMO_DATA_STORE_URL}/v1/datastore/namespaces/{NMS_NAMESPACE}")
print(f"Data Store - Status Code: {response.status_code}\nResponse JSON: {response.json()}")

# Verify Namespace in Entity Store
namespace_obj = entity_client.namespaces.retrieve(namespace_id=NMS_NAMESPACE)
print(f"\nEntity Store - Namespace: {namespace_obj.id}")
print(f"Created at: {namespace_obj.created_at}")
print(f"Description: {namespace_obj.description}")
print(f"Project: {namespace_obj.project}")

HTTP Request: GET http://nemo-entity-store.nemo.svc.cluster.local:8000/v1/namespaces/xlam-tutorial-ns "HTTP/1.1 200 OK"


Data Store - Status Code: 201
Response JSON: {'namespace': 'xlam-tutorial-ns', 'created_at': '2025-08-25T04:43:44Z', 'updated_at': '2025-08-25T04:43:44Z'}

Entity Store - Namespace: xlam-tutorial-ns
Created at: 2025-08-25 04:43:44.829989
Description: None
Project: None


**Tips**:
To list all available namespaces use
```python
requests.get(f"{NDS_URL}/v1/datastore/namespaces/") # For Data Store
nemo_client.namespaces.list() # For Entity Store
```

To delete a namespace use:
```python
requests.delete(f"{NDS_URL}/v1/datastore/namespaces/{namespace}") # For Data Store
nemo_client.namespaces.delete(namespace) # For Entity Store
```

---
<a id="step-1"></a>
## Step 1: Upload Data to NeMo Data Store

The NeMo Data Store supports data management using the Hugging Face `HfApi` Client. 

**Note that this step does not interact with Hugging Face at all, it just uses the client library to interact with NeMo Data Store.** This is in comparison to the previous notebook, where we used the `load_dataset` API to download the xLAM dataset from Hugging Face's repository.

More information can be found in [documentation](https://docs.nvidia.com/nemo/microservices/latest/manage-entities/tutorials/manage-dataset-files.html#set-up-hugging-face-client-with-nemo-data-store)

### 1.1 Create Repository

In [7]:
repo_id = f"{NMS_NAMESPACE}/{DATASET_NAME}"

In [8]:
from huggingface_hub import HfApi

hf_api = HfApi(endpoint=f"{NEMO_DATA_STORE_URL}/v1/hf", token="")

# Create repo
hf_api.create_repo(
    repo_id=repo_id,
    repo_type='dataset',
)

RepoUrl('datasets/xlam-tutorial-ns/xlam-ft-dataset', endpoint='http://nemo-data-store.nemo.svc.cluster.local:3000/v1/hf', repo_type='dataset', repo_id='xlam-tutorial-ns/xlam-ft-dataset')

Next, creating a dataset programmatically requires two steps: uploading and registration. More information can be found in [documentation](https://docs.nvidia.com/nemo/microservices/latest/manage-entities/datasets/create-dataset.html).

### 1.2 Upload Dataset Files to NeMo Data Store

In [9]:
hf_api.upload_file(path_or_fileobj=train_fp,
    path_in_repo="training/training.jsonl",
    repo_id=repo_id,
    repo_type='dataset',
)

hf_api.upload_file(path_or_fileobj=val_fp,
    path_in_repo="validation/validation.jsonl",
    repo_id=repo_id,
    repo_type='dataset',
)

hf_api.upload_file(path_or_fileobj=test_fp,
    path_in_repo="testing/xlam-test-single.jsonl",
    repo_id=repo_id,
    repo_type='dataset',
)

training.jsonl:   0%|          | 0.00/6.06M [00:00<?, ?B/s]

validation.jsonl:   0%|          | 0.00/1.30M [00:00<?, ?B/s]

xlam-test-single.jsonl:   0%|          | 0.00/1.19M [00:00<?, ?B/s]

CommitInfo(commit_url='', commit_message='Upload testing/xlam-test-single.jsonl with huggingface_hub', commit_description='', oid='d1b01e00c921c2f2cd85a78a96652ea94eca692e', pr_url=None, repo_url=RepoUrl('', endpoint='https://huggingface.co', repo_type='model', repo_id=''), pr_revision=None, pr_num=None)

Other tips:
* Take a look at the `path_in_repo` argument above. If there are more than one files in the subfolders:
    * All the .jsonl files in `training/` will be merged and used for training by customizer.
    * All the .jsonl files in `validation/` will be merged and used for validation by customizer.
* NeMo Data Store generally supports data management using the [HfApi API](https://huggingface.co/docs/huggingface_hub/en/package_reference/hf_api). For example, to delete a repo, you may use - 
```python
   hf_api.delete_repo(
     repo_id=repo_id,
     repo_type="dataset"
)
```

### 1.3 Register the Dataset with NeMo Entity Store

To use a dataset for operations such as evaluations and customizations, register a dataset using the `nemo_client.datasets.create()` method.
Register the dataset to refer to it by its namespace and name afterward.

In [10]:
# Create dataset
dataset = entity_client.datasets.create(
    name=DATASET_NAME,
    namespace=NMS_NAMESPACE,
    description="Tool calling xLAM dataset in OpenAI ChatCompletions format",
    files_url=f"hf://datasets/{NMS_NAMESPACE}/{DATASET_NAME}",
    project="tool_calling",
)
print(f"Created dataset: {dataset.namespace}/{dataset.name}")
dataset

HTTP Request: POST http://nemo-entity-store.nemo.svc.cluster.local:8000/v1/datasets "HTTP/1.1 200 OK"


Created dataset: xlam-tutorial-ns/xlam-ft-dataset


Dataset(files_url='hf://datasets/xlam-tutorial-ns/xlam-ft-dataset', id='dataset-6vSfQ1eF24q2AEEWXz3w1C', created_at=datetime.datetime(2025, 8, 25, 4, 44, 2, 789781), custom_fields={}, description='Tool calling xLAM dataset in OpenAI ChatCompletions format', format=None, hf_endpoint=None, limit=None, name='xlam-ft-dataset', namespace='xlam-tutorial-ns', project='tool_calling', split=None, updated_at=datetime.datetime(2025, 8, 25, 4, 44, 2, 789784))

In [11]:
# Sanity check to validate dataset
dataset_obj = entity_client.datasets.retrieve(namespace=NMS_NAMESPACE, dataset_name=DATASET_NAME)

print("Files URL:", dataset_obj.files_url)
assert dataset_obj.files_url == f"hf://datasets/{repo_id}"

HTTP Request: GET http://nemo-entity-store.nemo.svc.cluster.local:8000/v1/datasets/xlam-tutorial-ns/xlam-ft-dataset "HTTP/1.1 200 OK"


Files URL: hf://datasets/xlam-tutorial-ns/xlam-ft-dataset


---
<a id="step-2"></a>
## 2. LoRA Customization with NeMo Customizer

### 2.1 Start the Training Job

Start the training job by calling `nemo_client.customization.jobs.create()` method.
The following code sets the training parameters and starts the job.

**The training job will take approximately 45 minutes to complete.**

In [12]:
# Create customization job
# If WANDB_API_KEY is set, we send it in the request header, which will report the training metrics to Weights & Biases (WandB).
if WANDB_API_KEY:
    client_with_wandb = customizer_client.with_options(default_headers={"wandb-api-key": WANDB_API_KEY})
else:
    client_with_wandb = customizer_client

customization = client_with_wandb.customization.jobs.create(
    name="llama-3.2-1b-xlam-ft",
    output_model=CUSTOM_MODEL,
    config=f"{BASE_MODEL}@{BASE_MODEL_VERSION}",
    dataset={"name": DATASET_NAME, "namespace": NMS_NAMESPACE},
    hyperparameters={
        "training_type": "sft",
        "finetuning_type": "lora",
        "epochs": 2,
        "batch_size": 16,
        "learning_rate": 0.0001,
        "lora": {
            "adapter_dim": 32,
            "adapter_dropout": 0.1
        }
    }
)
print(f"Created customization job: {customization.id}")
customization

HTTP Request: POST http://nemo-customizer.nemo.svc.cluster.local:8000/v1/customization/jobs "HTTP/1.1 409 Conflict"
Retrying request to /v1/customization/jobs in 0.390251 seconds
HTTP Request: POST http://nemo-customizer.nemo.svc.cluster.local:8000/v1/customization/jobs "HTTP/1.1 409 Conflict"
Retrying request to /v1/customization/jobs in 0.754712 seconds
HTTP Request: POST http://nemo-customizer.nemo.svc.cluster.local:8000/v1/customization/jobs "HTTP/1.1 200 OK"


Created customization job: cust-k3VDbkH2gWueqL1u8MunA


CustomizationJob(config='meta/llama-3.2-1b-instruct@v1.0.0+L40', dataset='xlam-tutorial-ns/xlam-ft-dataset', hyperparameters=Hyperparameters(finetuning_type='lora', batch_size=16, distillation=None, dpo=None, epochs=2, learning_rate=0.0001, log_every_n_steps=None, lora=LoraParameters(adapter_dim=32, adapter_dropout=0.1, alpha=16, target_modules=None), sequence_packing_enabled=False, sft=None, training_type='sft', val_check_interval=None, weight_decay=None), id='cust-k3VDbkH2gWueqL1u8MunA', config_snapshot=CustomizationConfigJobValue(base_model='meta/llama-3.2-1b-instruct', max_seq_length=4096, precision='bf16-mixed', training_option=CustomizationTrainingOption(finetuning_type='lora', micro_batch_size=1, num_gpus=1, training_type='sft', data_parallel_size=1, num_nodes=1, pipeline_parallel_size=1, tensor_parallel_size=1, use_sequence_parallel=False), dataset_schema=None, prompt_template='{prompt} {completion}'), created_at=datetime.datetime(2025, 8, 25, 4, 44, 13, 342910), dataset_parame

**Note**: In the snippet above, the model name and version are passed directly in the `config` argument. However, in production environments, administrators typically create customization **[targets](https://docs.nvidia.com/nemo/microservices/latest/fine-tune/manage-customization-targets/index.html)** and corresponding **[configs](https://docs.nvidia.com/nemo/microservices/latest/fine-tune/manage-customization-configs/index.html)**. This approach allows you to configure once and reuse model configurations for multiple customization jobs. In such cases, you simply reference the created configuration in the `config` argument. For more details, refer to the documentation.

The following code sets variables for storing the job ID and customized model name.

In [13]:
# To track status
JOB_ID = customization.id

customization = customizer_client.customization.jobs.retrieve(JOB_ID)

# This will be the name of the model that will be used to send inference queries to
CUSTOMIZED_MODEL = customization.output_model

HTTP Request: GET http://nemo-customizer.nemo.svc.cluster.local:8000/v1/customization/jobs/cust-k3VDbkH2gWueqL1u8MunA "HTTP/1.1 200 OK"


**Tips**:
* If you configured the NeMo Customizer microservice with your own [Weights & Biases (WandB)](https://wandb.ai/) API key, you can find the training graphs and logs in your WandB account, "nvidia-nemo-customizer" project. Your run ID is similar to your customization `JOB_ID`.
  
* To cancel a job that you scheduled incorrectly, run the following code.
  
  ```python
  nemo_client.customization.jobs.cancel(job_id=JOB_ID)
  ```

### 2.2 Get Job Status

Get the job status by using the `nemo_client.customization.jobs.status()` method.
The following code sets the job ID and sends the request.

In [14]:
# Get job status
job_status = customizer_client.customization.jobs.status(job_id=JOB_ID)

print("Percentage done:", job_status.percentage_done)
print("Job Status:", json.dumps(job_status.model_dump(), indent=2, default=str))

HTTP Request: GET http://nemo-customizer.nemo.svc.cluster.local:8000/v1/customization/jobs/cust-k3VDbkH2gWueqL1u8MunA/status "HTTP/1.1 200 OK"


Percentage done: 0.0
Job Status: {
  "created_at": "2025-08-25 04:44:17.537483",
  "status": "pending",
  "updated_at": "2025-08-25 04:44:17.537483",
  "best_epoch": null,
  "elapsed_time": 0.0,
  "epochs_completed": 0,
  "metrics": null,
  "percentage_done": 0.0,
  "status_logs": [
    {
      "updated_at": "2025-08-25 04:44:17.537483",
      "detail": null,
      "message": "created"
    },
    {
      "updated_at": "2025-08-25 04:44:17.537483",
      "detail": "The training job is pending",
      "message": "TrainingJobPending"
    }
  ],
  "steps_completed": 0,
  "steps_per_epoch": null,
  "train_loss": null,
  "val_loss": null
}


In [16]:
# Add wait job function to wait for the customization job to complete

from time import sleep, time

def wait_job(nemo_client, job_id: str, polling_interval: int = 10, timeout: int = 6000):
    """Helper for waiting an eval job using SDK."""
    start_time = time()
    job = nemo_client.customization.jobs.retrieve(job_id=job_id)
    status = job.status

    while (status in ["pending", "created", "running"]):
        # Check for timeout
        if time() - start_time > timeout:
            raise RuntimeError(f"Took more than {timeout} seconds.")

        # Sleep before polling again
        sleep(polling_interval)

        # Fetch updated status and progress
        job = nemo_client.customization.jobs.retrieve(job_id=job_id)
        status = job.status
        progress = 0.0
        if status == "running" and job.status_details:
            progress = job.status_details.percentage_done or 0.0
        elif status == "completed":
            progress = 100

        print(f"Job status: {status} after {time() - start_time:.2f} seconds. Progress: {progress}%")


    return job

job = wait_job(customizer_client, JOB_ID, polling_interval=5, timeout=2400)

# Wait for 2 minutes, because sometimes, the job is finished, but the finetuned model is not ready in NIM yet.
sleep(120)

HTTP Request: GET http://nemo-customizer.nemo.svc.cluster.local:8000/v1/customization/jobs/cust-k3VDbkH2gWueqL1u8MunA "HTTP/1.1 200 OK"
HTTP Request: GET http://nemo-customizer.nemo.svc.cluster.local:8000/v1/customization/jobs/cust-k3VDbkH2gWueqL1u8MunA "HTTP/1.1 200 OK"


Job status: running after 5.04 seconds. Progress: 0.0%


HTTP Request: GET http://nemo-customizer.nemo.svc.cluster.local:8000/v1/customization/jobs/cust-k3VDbkH2gWueqL1u8MunA "HTTP/1.1 200 OK"


Job status: running after 10.06 seconds. Progress: 0.0%


HTTP Request: GET http://nemo-customizer.nemo.svc.cluster.local:8000/v1/customization/jobs/cust-k3VDbkH2gWueqL1u8MunA "HTTP/1.1 200 OK"


...

Job status: running after 773.22 seconds. Progress: 100.0%


HTTP Request: GET http://nemo-customizer.nemo.svc.cluster.local:8000/v1/customization/jobs/cust-k3VDbkH2gWueqL1u8MunA "HTTP/1.1 200 OK"


Job status: running after 778.45 seconds. Progress: 100.0%


HTTP Request: GET http://nemo-customizer.nemo.svc.cluster.local:8000/v1/customization/jobs/cust-k3VDbkH2gWueqL1u8MunA "HTTP/1.1 200 OK"


Job status: completed after 783.47 seconds. Progress: 100%


**IMPORTANT:** At this point, the customization job should be completed. If waiting for the job to finish failed or the status is not `"completed"`, please check the logs (`job.status_details.status_logs`).

### 2.3 Validate Availability of Custom Model
The following NeMo Entity Store API should display the model when the training job is complete.
The list below shows all models filtered by your namespace and sorted by the latest first.
For more information about this API, see the [NeMo Entity Store API reference](https://docs.nvidia.com/nemo/microservices/latest/api/entity-store.html).
With the following code, you can find all customized models, including the one trained in the previous cells.
Look for the `name` fields in the output, which should match your `CUSTOMIZED_MODEL`.

In [17]:
# List models with filters
models_page = entity_client.models.list(
    filter={"namespace": NMS_NAMESPACE},
    sort="-created_at"
)

# Print models information
print(f"Found {len(models_page.data)} models in namespace {NMS_NAMESPACE}:")
for model in models_page.data:
    print(f"\nModel: {model.name}")
    print(f"  Namespace: {model.namespace}")
    print(f"  Base Model: {model.base_model}")
    print(f"  Created: {model.created_at}")
    if model.peft:
        print(f"  Fine-tuning Type: {model.peft.finetuning_type}")

HTTP Request: GET http://nemo-entity-store.nemo.svc.cluster.local:8000/v1/models?filter%5Bnamespace%5D=xlam-tutorial-ns&sort=-created_at "HTTP/1.1 200 OK"


Found 1 models in namespace xlam-tutorial-ns:

Model: llama-3.2-1b-xlam-run1@v1
  Namespace: xlam-tutorial-ns
  Base Model: meta/llama-3.2-1b-instruct
  Created: 2025-08-25 04:44:17.914143
  Fine-tuning Type: lora


 The customized model can also be retrieved directly by using its name.

In [18]:
# CUSTOMIZED_MODEL is constructed as `namespace/model_name`, so we need to extract the model name
model = entity_client.models.retrieve(namespace=NMS_NAMESPACE, model_name=CUSTOMIZED_MODEL.split("/")[1])

print(f"Model: {model.namespace}/{model.name}")
print(f"Base Model: {model.base_model}")
print(f"Status: {model.artifact.status}")

HTTP Request: GET http://nemo-entity-store.nemo.svc.cluster.local:8000/v1/models/xlam-tutorial-ns/llama-3.2-1b-xlam-run1@v1 "HTTP/1.1 200 OK"


Model: xlam-tutorial-ns/llama-3.2-1b-xlam-run1@v1
Base Model: meta/llama-3.2-1b-instruct
Status: upload_completed


NVIDIA NIM directly picks up the LoRA adapters from NeMo Entity Store. You can also query the NIM endpoint to look for it, as shown in the following code.

In [19]:
# Check if the custom LoRA model is hosted by NVIDIA NIM
models = entity_client.inference.models.list()
model_names = [model.id for model in models.data]

assert CUSTOMIZED_MODEL in model_names, \
    f"Model {CUSTOMIZED_MODEL} not found"

HTTP Request: GET http://nemo-nim-proxy.nemo.svc.cluster.local:8000/v1/models "HTTP/1.1 200 OK"


In [20]:
print(model_names)

['meta/llama-3.2-1b-instruct', 'xlam-tutorial-ns/llama-3.2-1b-xlam-run1@v1']


---

<a id="step-3"></a>
## Step 3: Sanity Test the Customized Model By Running Sample Inference

Once the model is customized, its adapter is automatically saved in NeMo Entity Store and is ready to be picked up by NVIDIA NIM.
You can test the model by sending a prompt to its NIM endpoint.

First, choose one of the examples from the test set.

### 3.1 Get Test Data Sample

In [21]:
def read_jsonl(file_path):
    """Reads a JSON Lines file and yields parsed JSON objects"""
    with open(file_path, 'r', encoding='utf-8') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespace
            if not line:
                continue  # Skip empty lines
            try:
                yield json.loads(line)
            except json.JSONDecodeError as e:
                print(f"Error decoding JSON: {e}")
                continue


test_data = list(read_jsonl(test_fp))

print(f"There are {len(test_data)} examples in the test set")

There are 713 examples in the test set


In [22]:
# Randomly choose
test_sample = random.choice(test_data)

# Visualize the inputs to the LLM - user query and available tools
test_sample['messages'], test_sample['tools']

([{'role': 'user',
   'content': 'Could you help me search for tech startups in Germany in German language, showing only the first 5 results?'}],
 [{'type': 'function',
   'function': {'name': 'search',
    'description': 'Performs a search query on the specified search engine using given parameters and returns the response as JSON or text.',
    'parameters': {'type': 'object',
     'properties': {'pageno': {'description': 'The page number of the search results.',
       'type': 'integer',
       'default': '1'},
      'country': {'description': 'The country code for the search localization.',
       'type': 'string',
       'default': 'US'},
      'lang': {'description': 'The language code for the search localization.',
       'type': 'string',
       'default': 'en'},
      'search': {'description': 'The search string to query.',
       'type': 'string',
       'default': 'Hello'},
      'perpage': {'description': 'The number of results per page. Defaults to 10.',
       'type': 'in

### 3.2 Send an Inference Call to NIM

NIM exposes an OpenAI-compatible completions API endpoint, which you can query using the `OpenAI` client library as shown in the following code.

In [23]:
inference_client = OpenAI(
  base_url = f"{NIM_URL}/v1",
  api_key = "None"
)

completion = inference_client.chat.completions.create(
  model = CUSTOMIZED_MODEL,
  messages = test_sample["messages"],
  tools = test_sample["tools"],
  tool_choice = 'auto',
  temperature = 0.1,
  top_p = 0.7,
  max_tokens = 512,
  stream = False
)

completion.choices[0].message.tool_calls

HTTP Request: POST http://nemo-nim-proxy.nemo.svc.cluster.local:8000/v1/chat/completions "HTTP/1.1 200 OK"


[ChatCompletionMessageFunctionToolCall(id='chatcmpl-tool-c0a5eb39e431487890f978d653e1aab2', function=Function(arguments='{"pageno": 1, "country": "DE", "lang": "de", "search": "tech startups in Germany", "perpage": 5}', name='search'), type='function')]

The Python SDK also supports the same inference call, as shown in the following code.

In [24]:
completion = entity_client.chat.completions.create(
  model = CUSTOMIZED_MODEL,
  messages = test_sample["messages"],
  tools = test_sample["tools"],
  tool_choice = 'auto',
  temperature = 0.1,
  top_p = 0.7,
  max_tokens = 512,
  stream = False
)

completion.choices[0].message.tool_calls

HTTP Request: POST http://nemo-nim-proxy.nemo.svc.cluster.local:8000/v1/chat/completions "HTTP/1.1 200 OK"


[ChatCompletionMessageToolCall(id='chatcmpl-tool-16d99f5b6f954e1f9f5897d1a737cc3a', function=Function(arguments='{"pageno": 1, "country": "DE", "lang": "de", "search": "tech startups in Germany", "perpage": 5}', name='search'), type='function')]

Given that the fine-tuning job was successful, you can get an inference result comparable to the ground truth:

In [25]:
# The ground truth answer
test_sample['tool_calls']

[{'type': 'function',
  'function': {'name': 'search',
   'arguments': {'pageno': 1,
    'country': 'DE',
    'lang': 'de',
    'search': 'Tech-Startups',
    'perpage': 5}}}]

**Note:** In production environments, application developers typically provide their own set of tools relevant to the specific task. The model must select from these tools based on the given query. To explore this further, you can sample a data point from the dataset to see which tools are available, then experiment by constructing a query and observing the model’s response.

### 3.3 Take Note of Your Custom Model Name

Take note of your custom model name, as you will use it to run evaluations in the subsequent notebook.

In [26]:
print(f"Name of your custom model is: {CUSTOMIZED_MODEL}")

Name of your custom model is: xlam-tutorial-ns/llama-3.2-1b-xlam-run1@v1
